In [1]:
import pandas as pd
import numpy as np
symptom=pd.read_csv('Symptom-severity.csv')
disease=pd.read_csv('symptom_Description.csv')
prec=pd.read_csv('symptom_precaution.csv')
med=pd.read_csv('medicine_disease.csv')
df=pd.read_csv('dataset.csv')


disease.head()

symptom.head()

prec.head()

df.head()

med.head()

# Merge 'medicine' and 'disease' data on 'Disease' column
merged_data = pd.merge( med,df, on='Disease',how="left")
merged_data = pd.merge( prec,merged_data, on='Disease',how="left")

# Display the merged data
print(merged_data.head())
df=merged_data
df.head()

         Disease     Precaution_1              Precaution_2      Precaution_3  \
0  Drug Reaction  stop irritation  consult nearest hospital  stop taking drug   
1  Drug Reaction  stop irritation  consult nearest hospital  stop taking drug   
2  Drug Reaction  stop irritation  consult nearest hospital  stop taking drug   
3  Drug Reaction  stop irritation  consult nearest hospital  stop taking drug   
4  Drug Reaction  stop irritation  consult nearest hospital  stop taking drug   

  Precaution_4  Unnamed: 0                                           Medicine  \
0    follow up         0.0  Medicine for Drug Reaction (consultation requi...   
1    follow up         0.0  Medicine for Drug Reaction (consultation requi...   
2    follow up         0.0  Medicine for Drug Reaction (consultation requi...   
3    follow up         0.0  Medicine for Drug Reaction (consultation requi...   
4    follow up         0.0  Medicine for Drug Reaction (consultation requi...   

  Symptom_1      Symptom_2

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4,Unnamed: 0,Medicine,Symptom_1,Symptom_2,Symptom_3,...,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up,0.0,Medicine for Drug Reaction (consultation requi...,itching,skin_rash,stomach_pain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up,0.0,Medicine for Drug Reaction (consultation requi...,itching,stomach_pain,burning_micturition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up,0.0,Medicine for Drug Reaction (consultation requi...,itching,skin_rash,burning_micturition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up,0.0,Medicine for Drug Reaction (consultation requi...,itching,skin_rash,stomach_pain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up,0.0,Medicine for Drug Reaction (consultation requi...,itching,skin_rash,stomach_pain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Replacing underscores with spaces in column names
df.columns = df.columns.str.replace('_', ' ')

# Stripping whitespace from the DataFrame values
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# No need to flatten and reshape as it might distort the original structure
# The above code assumes that the intention is to strip leading/trailing spaces from string columns.


df.fillna(0,inplace=True)

k= df['Disease'].unique()
k=pd.DataFrame(k)
k.rename(columns={k.columns[0]: 'Disease'})

symptom['Symptom'] = symptom['Symptom'].str.replace('_',' ')
symptom.head()

len(symptom['Symptom'].unique())

symptom.drop_duplicates(inplace=True)
len(symptom['Symptom'].unique())

df.drop_duplicates(subset='Disease', inplace=True)

# Aggregate duplicate entries in df_symptom_weights
df_symptom_weights =symptom.groupby('Symptom', as_index=False)['weight'].mean()
df['Disease'].drop_duplicates(inplace=True)
# Reshape the data for collaborative filtering matrix
coll_matrix = df.melt(id_vars='Disease', value_vars=df.columns[1:],
                                       value_name='Symptom') \
    .merge(df_symptom_weights, on='Symptom') \
    .pivot(index='Disease', columns='Symptom', values='weight')
coll_matrix

coll_matrix.fillna(0,inplace=True)
coll_matrix.head()
t = coll_matrix.transpose()
t.head()

# gives similarty of each book based on and its distances
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(t)

similarity_scores

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.17149859],
       [0.        , 1.        , 0.        , ..., 0.40824829, 0.        ,
        0.24253563],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.40824829, 0.        , ..., 1.        , 0.        ,
        0.39605902],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.24253563],
       [0.17149859, 0.24253563, 0.        , ..., 0.39605902, 0.24253563,
        1.        ]])

In [3]:
merged_data.columns

Index(['Disease', 'Precaution 1', 'Precaution 2', 'Precaution 3',
       'Precaution 4', 'Unnamed: 0', 'Medicine', 'Symptom 1', 'Symptom 2',
       'Symptom 3', 'Symptom 4', 'Symptom 5', 'Symptom 6', 'Symptom 7',
       'Symptom 8', 'Symptom 9', 'Symptom 10', 'Symptom 11', 'Symptom 12',
       'Symptom 13', 'Symptom 14', 'Symptom 15', 'Symptom 16', 'Symptom 17'],
      dtype='object')

In [ ]:
def recommend(symptoms):
    indices = []
    for symptom in symptoms:
        if symptom in t.index:
            indices.append(np.where(t.index == symptom)[0][0])

    if not indices:
        return ["No matching symptoms found. Please check your input."]

    combined_weights = np.zeros(coll_matrix.shape[0])

    for index in indices:
        combined_weights += coll_matrix.iloc[:, index]

    # Normalize combined weights to have values between 0 and 1
    max_weight = max(combined_weights)
    if max_weight > 0:
        combined_weights /= max_weight

    common_diseases = round((combined_weights * 100), 2)
    common_diseases = common_diseases.sort_values(ascending=False)
    common_diseases = common_diseases[common_diseases >= 15]
    top_diseases = common_diseases.index[0:3]  # Top 3 predicted diseases

    # Retrieve corresponding medicines for top predicted diseases
    corresponding_medicines = []
    for disease in top_diseases:
        # Fetch unique medicines for each disease
        medicines_for_disease = merged_data.loc[merged_data['Disease'] == disease, 'Medicine'].unique()
        prec1 = merged_data.loc[merged_data['Disease'] == disease, 'Precaution 1'].unique()
        prec2 = merged_data.loc[merged_data['Disease'] == disease, 'Precaution 2'].unique()
        prec3 = merged_data.loc[merged_data['Disease'] == disease, 'Precaution 3'].unique()
        prec4 = merged_data.loc[merged_data['Disease'] == disease, 'Precaution 4'].unique()

        corresponding_medicines.append((disease, medicines_for_disease,prec1,prec2,prec3,prec4))

    return corresponding_medicines

In [5]:
recommend(['chills','shivering'])

[('Allergy',
  array(['Cetirizine (Zyrtec)'], dtype=object),
  array(['apply calamine'], dtype=object),
  array(['cover area with bandage'], dtype=object),
  array([nan], dtype=object),
  array(['use ice to compress itching'], dtype=object)),
 ('Dengue',
  array(['Supportive care, hydration'], dtype=object),
  array(['drink papaya leaf juice'], dtype=object),
  array(['avoid fatty spicy food'], dtype=object),
  array(['keep mosquitos away'], dtype=object),
  array(['keep hydrated'], dtype=object)),
 ('Malaria',
  array(['Artemether-Lumefantrine (Coartem)'], dtype=object),
  array(['Consult nearest hospital'], dtype=object),
  array(['avoid oily food'], dtype=object),
  array(['avoid non veg food'], dtype=object),
  array(['keep mosquitos out'], dtype=object))]

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import scrolledtext
from PIL import Image, ImageTk
from ttkthemes import ThemedStyle

# Function to handle button click event and update the scrolledtext with suggestions
def show_suggestions():
    symptoms = entry.get().split(',')
    suggestions = recommend(symptoms)
    suggestion_text.delete(1.0, tk.END)
    for suggestion in suggestions:
        suggestion_text.insert(tk.END, suggestion + "\n")

# Main Tkinter application
root = tk.Tk()
root.title("Disease Recommender App")
root.geometry("800x600")  # Set initial window size

# Load and resize the background image to fit the screen size
# bg_image = Image.open("image.jpg")  # Replace "background.jpg" with your image file
# bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.ANTIALIAS)
# bg_photo = ImageTk.PhotoImage(bg_image)
# # Set up background image
# bg_label = tk.Label(root, image=bg_photo)
# bg_label.place(relwidth=1, relheight=1)

# Set up styles for themed widgets
style = ThemedStyle(root)
style.set_theme("radiance")  # You can choose other themes like "clearlooks" or "equilux"

# Heading
heading_label = ttk.Label(root, text="Disease Recommender", font=("Arial", 24, "bold"))
heading_label.pack(pady=20)

# Entry widget for user to type book name
entry = ttk.Entry(root, font=("Arial", 14))
entry.pack(pady=10, padx=20, ipadx=10, ipady=10, fill=tk.X)

# Button to get suggestions
suggest_button = ttk.Button(root, text="Suggest", command=show_suggestions)
suggest_button.pack(pady=10)

# Scrolled Text to display suggestions
suggestion_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=10, font=("Arial", 12))
suggestion_text.pack(padx=20, pady=20, fill=tk.BOTH, expand=True)

# Custom scrollbar for suggestions
scrollbar = ttk.Scrollbar(suggestion_text, orient="vertical", command=suggestion_text.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
suggestion_text.config(yscrollcommand=scrollbar.set)

# Start the main loop
root.mainloop()


In [ ]:
pip install ttkthemes
